# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846897266400                   -0.70    4.8
  2   -7.852327958634       -2.27       -1.53    1.0
  3   -7.852646054793       -3.50       -2.52    3.0
  4   -7.852646678824       -6.20       -3.37    2.8
  5   -7.852646686098       -8.14       -4.81    1.2
  6   -7.852646686725       -9.20       -5.28    3.5
  7   -7.852646686730      -11.29       -6.15    1.8
  8   -7.852646686730      -12.71       -7.30    2.2
  9   -7.852646686730   +    -Inf       -7.48    2.0
 10   -7.852646686730   +    -Inf       -8.51    2.2
 11   -7.852646686730   +    -Inf       -9.46    2.2
 12   -7.852646686730   +    -Inf      -10.52    2.2
 13   -7.852646686730   +    -Inf      -10.77    2.0
 14   -7.852646686730   +  -14.75      -11.91    2.2
 15   -7.852646686730      -14.57      -12.65    2.5


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846872976963                   -0.70           4.5
  2   -7.852525586223       -2.25       -1.64   0.80    2.2
  3   -7.852635832994       -3.96       -2.73   0.80    1.0
  4   -7.852646540379       -4.97       -3.25   0.80    2.0
  5   -7.852646676032       -6.87       -4.10   0.80    1.5
  6   -7.852646686325       -7.99       -4.80   0.80    1.5
  7   -7.852646686718       -9.41       -5.52   0.80    2.2
  8   -7.852646686729      -10.94       -6.57   0.80    1.8
  9   -7.852646686730      -12.39       -7.26   0.80    2.2
 10   -7.852646686730      -14.35       -8.26   0.80    1.2
 11   -7.852646686730   +  -14.75       -8.34   0.80    2.2
 12   -7.852646686730   +    -Inf       -9.80   0.80    1.0
 13   -7.852646686730   +    -Inf       -9.83   0.80    3.2
 14   -7.852646686730   +  -15.05      -10.76   0.80    1.0
 15   -7.852646686730   +    -Inf      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.401985e+01     3.459275e+00
 * time: 0.3973841667175293
     1     1.337399e+00     1.763907e+00
 * time: 0.601402997970581
     2    -2.012057e+00     2.040344e+00
 * time: 0.6248941421508789
     3    -3.944220e+00     1.822190e+00
 * time: 0.6587369441986084
     4    -5.510506e+00     1.570311e+00
 * time: 0.6920671463012695
     5    -7.036076e+00     1.035605e+00
 * time: 0.7257411479949951
     6    -7.214904e+00     1.303708e+00
 * time: 0.7495560646057129
     7    -7.668802e+00     7.195014e-01
 * time: 0.7734489440917969
     8    -7.762098e+00     6.236794e-01
 * time: 0.7973861694335938
     9    -7.813468e+00     1.960155e-01
 * time: 0.8218679428100586
    10    -7.833277e+00     6.556674e-02
 * time: 0.8455901145935059
    11    -7.844446e+00     9.507543e-02
 * time: 0.8691730499267578
    12    -7.846101e+00     6.149073e-02
 * time: 0.8928680419921875
    13    -7.846223e+00     5.678739e-02
 * time: 0.91635394096

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846866035963                   -0.70    4.5
  2   -7.852318480066       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686711                   -2.55
  2   -7.852646686730      -10.72       -5.99
  3   -7.852646686730      -14.75      -12.69


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.502593681235752e-14
|ρ_newton - ρ_scfv| = 3.445684505969634e-13
|ρ_newton - ρ_dm|   = 3.141299473831221e-10
